Chapitre 12 - Créer une API
===

## A ? P ? I ?

### Quoi ?

Une API (Application Programming Interface) est un système informatique prévu pour la communication de données ou de fonctions à des services tiers. L'équipe de développement prévoit ainsi un moyen externe de réaliser des transactions depuis l'extérieur sans avoir à accéder au code interne d'un projet ou à ses données au format brut.

Les exemples d'API sont nombreux et touchent aussi bien le Web que le logiciel et les librairies de développement. Pour donner un exemple, on trouve des APIs servant des données (IIIF et les métadonnées d'image par exemple, OAI PMH et les métadonnées de catalogue, etc.) ou bien même des fonctions telles que Google Translate ([Documentation](https://cloud.google.com/translate/docs/quickstart)) ou d'autres telles que [LemLat](http://cophilab.ilc.cnr.it:8080/LatMorphWebApp/services/complete/arma,cano,domus) qui fournit l'analyse morphologique de mots latins.

Cependant, toutes les APIs ne sont pas forcément produites pour leur consommation par des tiers. Dans beaucoup de cas, il sera intéressant pour un projet de pouvoir fournir un moyen de communication entre le client et le serveur une fois les données chargées. 

![Autocomplete](images/autocomplete.gif)


Un exemple très commun serait l'*autocomplétion* d'un formulaire de recherche. Quand l'utilisateur tape quelques lettres, un script du côté client ira communiquer avec une page de l'API pour récupérer les données sans rafraichir la page actuelle.

### Comment ?

La consommation de données est bien évidemment le point le plus complexe d'une API: on voudra s'arranger un maximum pour que les données fournies soit faciles à extraire. Prenons deux exemples : une page HTML et un fichier JSON qui contiennent les mêmes données.

![Requête sur une version en HTML](./images/latin.html.png)

L'XPath correspondant à la deuxième traduction de `cano` est : `/html/body/div[1]/div/section/div/div[2]/ul[3]/li[2]`. On peut boucler sur `/html/body/div[1]/div/section/div/div[2]/ul` puis sur les `li` mais il est impossible, sans expression régulière, de diviser les traductions des analyses.

![Requête sur une version en JSON](./images/latin.json.png)

Le chemin JSON correspondant est `["results"][2]` où l'on trouve les analyses en dans une liste telle que `["results"][2]["analysis"][0]["lemmas"][0]["lemma"]` est égal à `cano`. Il est aussi très facile de boucler dessus.

| Élément de comparaisons | HTML   | JSON   |
| ----------------------- | ------ | ------ |
| Poids                   | 7364 o | 5257 o (Mais avec plus d'informations propres à la requête) |
| Parsage                 | Capable d'échouer si une page web est mal construire dans son header | Très peu probable d'échouer |
| Disponibilités dans les langages | Moyenne (librairies externes souvent) | Élevée |

Il va s'en dire que, d'abord pour le poids et la facilité de lecture machine, **le JSON est le plus abordable**. Il existe d'autres formats acceptable (XML par exemple) mais pour toute donnée structurée autre que du texte balisé, il est recommandé de passer par du JSON (Sur ce point : *[Mobile Performance testing JSON vs XML (2016)](https://www.infragistics.com/community/blogs/b/torrey-betts/posts/mobile-performance-testing-json-vs-xml), [JSON vs XML: A Comparative Performance Analysis  of Data Exchange Formats](http://ijcsn.org/IJCSN-2014/3-4/JSON-vs-XML-A-Comparative-Performance-Analysis-of-Data-Exchange-Formats.pdf), [JSON ou XML, quel format choisir? ](https://www.scriptol.fr/ajax/json-xml.php)*)

## Le gazetteer en données ouvertes

Nous allons chercher à partager nos données directement avec les utilisateurs. Nous allons prévoir deux visualisations des données : une page de recherche et une page pour un objet particulier.

### jsonify 

#### Flask et les Response

Votre premier réflexe sera sûrement d'utiliser **`json.dumps()`**. Cela montre que vous avez compris le cour. Cependant... **cela ne sera pas suffisant.**

Rappelez-vous, sur le web, une réponse est basée sur trois paramètres :
- les headers
- le code http
- le corps

Or, si l'on fait `return json.dumps()` nous ne retournons qu'une chaîne de caractères... Mais alors, que faisait `render_template` me demanderez-vous ?

En fait, `render_template()` ne renvoit pas une chaîne de caractère (c'était prévisible...) mais un objet `Response` qui est constitué d'un corps, d'un code http et d'un headers. Et vu que vous envoyez un template en html, la réponse dit "bon, le code par défaut, c'est *200* et les headers, on va prévenir que le mimetype du corps est *html*".

Et de fait, Flask, pour fonctionner véritablement correctement, a besoin de `Reponse`. Une [`Response`](http://flask.pocoo.org/docs/0.12/api/#response-objects) a 2 paramètres qui nous importent:
- `.headers` qui se comportent comme un dictionnaire
- `.status_code` qui se comportent comme un entier

de tel manière que l'on peut écrire :

```python
from flask import Flask, Response

app = Flask("Nom")

@app.route("/404")
def erreur_404():
    response = Response("Il y a eu une erreur")
    response.headers["content-type"] = "text/plain"
    response.status_code = 404
    return response
```

ou bien encore

```python
from flask import Flask, Response
import json
app = Flask("Nom")

@app.route("/du_json")
def une_route():
    mon_dictionnaire = {"une_cle" : "une valeur"}
    mon_json = json.dumps(mon_dictionnaire)
    # Juste une autre manière d'écrire ce qui a été écrit au-dessus
    response = Response(response, status=200, mimetype="application/json")
    return response
```

#### La méthode rapide

Tout développeur-se étant un-e fainéant-e intelligent-e (ou inversement), l'équipe derrière Flask a ajouté une fonction un peu raccourci pour le même résultat que le dernier exemple : `flask.jsonify()` :

```python
from flask import Flask, jsonify
import json
app = Flask("Nom")

@app.route("/du_json")
def une_route():
    mon_dictionnaire = {"une_cle" : "une valeur"}
    return jsonify(mon_dictionnaire)
```

et vu que jsonify retourne une `Response`, on peut aussi changer le code d'erreur :

```python
from flask import Flask, jsonify
import json
app = Flask("Nom")

@app.route("/erreur404")
def une_route():
    mon_dictionnaire = {"message" : "Vous avez une belle erreur ici !"}
    response = jsonify(mon_dictionnaire)
    response.status_code = 404
    return response
```

Plutôt simple non ?

### Tout n'est pas `jsonifiable` de base....

Nous avons vu jusque là qu'il était facile de convertir dictionnaires, listes, booléens, nombres, `None` et autres types très simple de Python. Qu'en est-il des données plus complexes telles que nous avons récupérées de notre base de données ?

`jsonify` étant basé sur `json.dumps()`, nous pouvons essayer `json.dumps()` sur nos objets :

In [ ]:
from modules_cours.gazetteer.modeles.donnees import Place
from modules_cours.gazetteer.app import app
from json import dumps

data = Place.query.get(1)
print(data)

In [ ]:
dumps(data)

`TypeError: <Place 1> is not JSON serializable` est normalement le retour final. `dump` et `dumps` ne savent pas gérer les objets que vous créez vous-mêmes. Comment faire alors ?

**La solution est simple:** on prévoit une fonction qui transformera nos objets en dictionnaires, liste, etc. Par exemple :

In [ ]:
def place_to_json(place):
    return {
        "nom": place.place_nom,
        "description": place.place_description,
        "longitude": place.place_longitude,
        "latitude": place.place_latitude
    }

print(dumps(place_to_json(data)))

Ultimement, on s'arrangera pour que la fonction soit facilement accessible en la transformant en méthode : 

```python
class Place(db.Model):
    place_id = db.Column(db.Integer, unique=True, nullable=False, primary_key=True, autoincrement=True)
    place_nom = db.Column(db.Text)
    place_description = db.Column(db.Text)
    place_longitude = db.Column(db.Float)
    place_latitude = db.Column(db.Float)
    place_type = db.Column(db.String(45))
    
    def to_dict(self):
        return {
            "nom": self.place_nom,
            "description": self.place_description,
            "longitude": self.place_longitude,
            "latitude": self.place_latitude
        }
```

### La page de recherche en JSON

Pour prévoir plus tard la capacité de créer un script pour faire de l'autocomplétion, nous allons proposer une route d'API permettant de faire une recherche plein texte.

Le code va être très proche du code de recherche original, simplement nous allons :
- proposer la navigation dans le json
- proposer le lien vers le résultat dans le Json

Pour gérer les différentes routes, nous avons divisé `routes.py` en `routes/general.py` et `routes/api.py`.

#### api.py

##### urllib

URLLib est un package python bien agréable quand il s'agit de travailler sur des URLs. Nous utilisons ici `urllib.parse.urlencode` afin d'encoder des paramètres `GET` via un dictionnaire. On pensera à ajouter `?` devant.

##### url_for(..., _external=True)

Le paramètre `_external=True` de `url_for()` permet de générer une URL absolue et non simplement une URL relative.


```python
from flask import render_template, request, url_for, jsonify
from urllib.parse import urlencode

from ..app import app
from ..constantes import LIEUX_PAR_PAGE, API_ROUTE
from ..modeles.donnees import Place

@app.route(API_ROUTE+"/places")
def api_places_browse():
    """ Route permettant la recherche plein-texte et la navigation classique

    On s'inspirera de http://jsonapi.org/ faute de pouvoir trouver temps d'y coller à 100%
    """
    # q est très souvent utilisé pour indiquer une capacité de recherche
    motclef = request.args.get("q", None)
    page = request.args.get("page", 1)

    if isinstance(page, str) and page.isdigit():
        page = int(page)
    else:
        page = 1

    if motclef:
        query = Place.query.filter(
            Place.place_nom.like("%{}%".format(motclef))
        )
    else:
        query = Place.query

    try:
        resultats = query.paginate(page=page, per_page=LIEUX_PAR_PAGE)
    except Exception:
        return Json_404()

    dict_resultats = {
        "links": {
            "self": request.url
        },
        "data": [
            place.to_jsonapi_dict()
            for place in resultats.items
        ]
    }

    if resultats.has_next:
        arguments = {
            "page": resultats.next_num
        }
        if motclef:
            arguments["q"] = motclef
        dict_resultats["links"]["next"] = url_for("api_places_browse", _external=True)+"?"+urlencode(arguments)

    if resultats.has_prev:
        arguments = {
            "page": resultats.prev_num
        }
        if motclef:
            arguments["q"] = motclef
        dict_resultats["links"]["prev"] = url_for("api_places_browse", _external=True)+"?"+urlencode(arguments)

    response = jsonify(dict_resultats)
    return response
```

#### Donnees.py

```python
from flask import url_for

class Place(db.Model):
    # ...

    def to_jsonapi_dict(self):
        """ It ressembles a little JSON API format but it is not completely compatible

        :return:
        """
        return {
            "type": "place",
            "id": self.place_id,
            "attributes": {
                "name": self.place_nom,
                "description": self.place_description,
                "longitude": self.place_longitude,
                "latitude": self.place_latitude,
                "category": self.place_type
            },
            "links": {
                "self": url_for("lieu", place_id=self.place_id, _external=True),
                "json": url_for("api_places_single", place_id=self.place_id, _external=True)
            },
            "relationships": {
                 "editions": [
                     author.author_to_json()
                     for author in self.authorships
                 ]
            }
        }
```

Codes issus de :
- [cours-flask/exemple17/gazetteer/modeles/donnees.py](cours-flask/exemple17/gazetteer/modeles/donnees.py)
- [cours-flask/exemple17/gazetteer/routes/api.py](cours-flask/exemple17/gazetteer/routes/api.py) 


### L'affichage en GEOJson

À partir de la documentation donnée ci-après, corriger le code suivant afin de fournir une vue en GeoJson :

(Dans exemple-17)
```python
@app.route(API_ROUTE+"/places/<place_id>")
def api_places_single(place_id):
    try:
        query = Place.query.get(place_id)
        return jsonify(query.to_jsonapi_dict())
    except:
        return Json_404()
```

- [GeoJson.org](http://geojson.org/)
- [Wikipedia : GeoJson](https://en.wikipedia.org/wiki/GeoJSON)
- [Exemple GeoJson Pleiades](https://pleiades.stoa.org/places/837078212/json)